In [1]:
from prepare_jpype import start_jpype
import pandas as pd
start_jpype()

from evaluation import test_train_loop_MOA, test_train_loop_RIVER, start_time_measuring, stop_time_measuring

MOA jar path location (config.ini): /Users/gomeshe/Dropbox/ciencia_computacao/dev/Using-MOA-API/moa.jar
JVM Location (system): 
/Users/gomeshe/Library/Java/JavaVirtualMachines/openjdk-20.0.1/Contents/Home
Sucessfully started the JVM and added MOA jar to the class path


# Experiments MOA vs River using ARF

* These tests were using the old test_train_loop() and run_experiment() methods. 

* They are safe to delete or update (but the river ones take too long to execute).

* **SAVED THE RESULTS BECAUSE RIVER TAKES TOO LONG TO EXECUTE! Especially for ARF100!**

## Experimental: creating a wrapper for ARF in python


In [2]:
from moa.classifiers.meta import AdaptiveRandomForest

class AdaptiveRandomForestWrapper:
    def __init__(self, stream_context, tree_learner=None, ensemble_size=100, 
                 m_features_mode=None, m_features_per_tree_size=60, lambda_param=6.0,
                number_of_jobs=1, drift_detection_method=None, warning_detection_method=None,
                disable_weighted_vote=False, disable_drift_detection=False, disable_background_learner=False):
        # Initialize instance attributes with default values        
        self.tree_learner = "(ARFHoeffdingTree -e 2000000 -g 50 -c 0.01)" if tree_learner is None else tree_learner
        self.ensemble_size = ensemble_size
        self.m_features_mode = "(Percentage (M * (m / 100)))" if m_features_mode is None else m_features_mode
        self.m_features_per_tree_size = m_features_per_tree_size
        self.lambda_param = lambda_param
        self.number_of_jobs = number_of_jobs
        self.drift_detection_method = "(ADWINChangeDetector -a 1.0E-3)" if drift_detection_method is None else drift_detection_method
        self.warning_detection_method = "(ADWINChangeDetector -a 1.0E-2)" if warning_detection_method is None else warning_detection_method
        self.disable_weighted_vote = disable_weighted_vote
        self.disable_drift_detection = disable_drift_detection
        self.disable_background_learner = disable_background_learner

        self.moa_learner = AdaptiveRandomForest()
        # self.moa_learner.getOptions().setViaCLIString(f"-l {self.tree_learner} -s {self.ensemble_size}" -o {self.m_features_mode} -m {self.m_features_per_tree_size} -a {self.lambda} -j {self.number_of_jobs} -x {self.drift_detection_method} -p {self.warning_detection_method} {'-w' if self.disable_weighted_vote else ''} {'-u' if self.disable_drift_detection else ''}  {'-q' if self.disable_background_learner else ''}")
        self.moa_learner.getOptions().setViaCLIString(f"-l {self.tree_learner} -s {self.ensemble_size} -o {self.m_features_mode} -m {self.m_features_per_tree_size} -a {self.lambda_param} -j {self.number_of_jobs} -x {self.drift_detection_method} -p {self.warning_detection_method} {'-w' if self.disable_weighted_vote else ''} {'-u' if self.disable_drift_detection else ''}  {'-q' if self.disable_background_learner else ''}")
    	# self.moa_learner.setRandomSeed(1)
        self.moa_learner.prepareForUse()
        self.moa_learner.resetLearningImpl()

        self.moa_learner.setModelContext(stream_context)

    def train(self, instance):
        self.moa_learner.trainOnInstance(instance)

    def predict(self, instance):
        return Utils.maxIndex(self.moa_learner.getVotesForInstance(instance))

    def predict_proba(self, instance):
        return self.moa_learner.getVotesForInstance(instance)

### Using the normal ```test_train_loop_MOA``` with the MOA ARF class

In [7]:
from moa.classifiers.meta import AdaptiveRandomForest
from moa.classifiers.trees import HoeffdingTree
from moa.core import Example
from moa.evaluation import BasicClassificationPerformanceEvaluator
from moa.streams.generators import RandomTreeGenerator

maxInstancesToProcess = 100
instancesProcessed = 1
sampleFrequency = 10

learner = AdaptiveRandomForest()
# Setting parameters using setViaCLIString
learner.getOptions().setViaCLIString("-s 10")
learner.resetLearningImpl()
learner.prepareForUse()

rtg = RandomTreeGenerator()
# Setting parameters using setViaCLIString
rtg.getOptions().setViaCLIString("-c 3 -u 10 -o 0")
rtg.prepareForUse()

# Setting parameters using the option attribute directly
evaluator = BasicClassificationPerformanceEvaluator()
evaluator.recallPerClassOption.set()
evaluator.prepareForUse()

learner.setModelContext(rtg.getHeader())

# Create empty lists to store the data
data = []
performance_names = []
performance_values = []

while rtg.hasMoreInstances() and instancesProcessed <= maxInstancesToProcess:
    trainInst = rtg.nextInstance()
    testInst = trainInst

    prediction = learner.getVotesForInstance(testInst)

    evaluator.addResult(testInst, prediction)
    learner.trainOnInstance(trainInst)

    if instancesProcessed == 1:
        performance_measurements = evaluator.getPerformanceMeasurements()
        performance_names = ["".join(measurement.getName()) for measurement in performance_measurements]

    if instancesProcessed % sampleFrequency == 0:
        performance_values = [measurement.getValue() for measurement in evaluator.getPerformanceMeasurements()]
        data.append(performance_values)
    
    instancesProcessed += 1

# Create a DataFrame using collected data
results_df = pd.DataFrame(data, columns=performance_names)

# Print the DataFrame
results_df

,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent),Recall for class 0 (percent),Recall for class 1 (percent),Recall for class 2 (percent)
0,10.0,30.000000,-16.666667,22.222222,-250.000000,33.333333,40.000000,0.0
1,20.0,40.000000,1.639344,14.285714,-50.000000,57.142857,44.444444,0.0
2,30.0,53.333333,18.287938,17.647059,-16.666667,66.666667,57.142857,0.0
3,40.0,62.500000,32.203390,28.571429,0.000000,78.947368,62.500000,0.0
4,50.0,68.000000,40.740741,44.827586,15.789474,80.000000,70.000000,0.0
5,60.0,71.666667,46.875000,51.428571,29.166667,83.333333,72.000000,0.0
6,70.0,75.714286,53.388171,58.536585,37.037037,86.486486,75.000000,0.0
7,80.0,76.250000,53.644404,58.695652,38.709677,88.372093,71.875000,0.0
8,90.0,76.666667,53.857422,59.615385,38.235294,90.000000,70.588235,0.0
9,100.0,77.000000,54.247066,60.344828,39.473684,91.071429,70.270270,0.0


### Using the ```experimental``` wrapper for the evaluator, but still a MOA object for learner (AdaptiveRandomForest)

In [8]:
from moa.classifiers.meta import AdaptiveRandomForest
from moa.classifiers.trees import HoeffdingTree
from moa.core import Example
from moa.core import Utils
from moa.evaluation import BasicClassificationPerformanceEvaluator
from moa.streams.generators import RandomTreeGenerator

maxInstancesToProcess = 100
instancesProcessed = 1
sampleFrequency = 10

learner = AdaptiveRandomForest()
# Setting parameters using setViaCLIString
learner.getOptions().setViaCLIString("-s 10")
learner.prepareForUse()

rtg = RandomTreeGenerator()
# Setting parameters using setViaCLIString
rtg.getOptions().setViaCLIString("-c 3 -u 10 -o 0")
rtg.prepareForUse()

# Creating the schema
schema = Schema(labels=[0,1,2])
# schema = Schema(moa_header=rtg.getHeader())
# Setting parameters using the option attribute directly
evaluator = ClassificationEvaluator(schema=schema, recallPerClass=True)

learner.setModelContext(rtg.getHeader())

# Create empty lists to store the data
data = []
performance_names = []
performance_values = []

while rtg.hasMoreInstances() and instancesProcessed <= maxInstancesToProcess:
    trainInst = rtg.nextInstance()
    testInst = trainInst

    prediction = learner.getVotesForInstance(testInst)

    evaluator.update(testInst.getData().classValue(), Utils.maxIndex(prediction))
    learner.trainOnInstance(trainInst)

    if instancesProcessed == 1:
        performance_names = evaluator.measurements()

    if instancesProcessed % sampleFrequency == 0:
        performance_values = evaluator.results()
        data.append(performance_values)
    
    instancesProcessed += 1

# Create a DataFrame using collected data
results_df = pd.DataFrame(data, columns=performance_names)

# Print the DataFrame
results_df

[0, 1, 2]


,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent),Recall for class 0 (percent),Recall for class 1 (percent),Recall for class 2 (percent)
0,10.0,30.000000,-16.666667,22.222222,-250.000000,33.333333,40.000000,0.0
1,20.0,40.000000,1.639344,14.285714,-50.000000,57.142857,44.444444,0.0
2,30.0,53.333333,18.287938,17.647059,-16.666667,66.666667,57.142857,0.0
3,40.0,62.500000,32.203390,28.571429,0.000000,78.947368,62.500000,0.0
4,50.0,68.000000,40.740741,44.827586,15.789474,80.000000,70.000000,0.0
5,60.0,71.666667,46.875000,51.428571,29.166667,83.333333,72.000000,0.0
6,70.0,75.714286,53.388171,58.536585,37.037037,86.486486,75.000000,0.0
7,80.0,76.250000,53.644404,58.695652,38.709677,88.372093,71.875000,0.0
8,90.0,76.666667,53.857422,59.615385,38.235294,90.000000,70.588235,0.0
9,100.0,77.000000,54.247066,60.344828,39.473684,91.071429,70.270270,0.0


In [14]:
schema = Schema(moa_header=rtg.getHeader())

print(schema.getLabelsIndexes())

AttributeError: 'Schema' object has no attribute 'getLabelsIndexes'

### Using the ```experimental``` wrapper for the evaluator and for the learner (AdaptiveRandomForestWrapper)

In [25]:
from moa.core import Utils

maxInstancesToProcess = 100
instancesProcessed = 1
sampleFrequency = 10

rtg = RandomTreeGenerator()
# Setting parameters using setViaCLIString
rtg.getOptions().setViaCLIString("-c 3 -u 10 -o 0")
rtg.prepareForUse()

learner = AdaptiveRandomForestWrapper(stream_context=rtg.getHeader(), 
                                      ensemble_size=10, 
                                      tree_learner="(ARFHoeffdingTree -e 2000000 -g 50 -c 0.01)")


# Setting parameters using the option attribute directly
rtg_labels = []
for g in rtg.getHeader().outputAttribute(1).getAttributeValues():
    rtg_labels.append(g)

print(type(rtg.getHeader().outputAttribute(1).getAttributeValues()))
print(type(rtg_labels))

evaluator = ClassificationEvaluator(labels=[0,1,2], recallPerClass=True)

# Create empty lists to store the data
data = []
performance_names = []
performance_values = []

while rtg.hasMoreInstances() and instancesProcessed <= maxInstancesToProcess:
    trainInst = rtg.nextInstance()
    testInst = trainInst

    prediction = learner.predict(testInst)

    evaluator.update(testInst.getData().classValue(), prediction)
    learner.train(trainInst)

    if instancesProcessed == 1:
        performance_names = evaluator.measurements()

    if instancesProcessed % sampleFrequency == 0:
        performance_values = evaluator.results()
        data.append(performance_values)

    instancesProcessed += 1

# Create a DataFrame using collected data
results_df = pd.DataFrame(data, columns=performance_names)
# Print the DataFrame
results_df


<java class 'moa.core.FastVector'>
<class 'list'>
[0, 1, 2]


,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent),Recall for class 0 (percent),Recall for class 1 (percent),Recall for class 2 (percent)
0,10.0,30.000000,-16.666667,22.222222,-250.000000,33.333333,40.000000,0.0
1,20.0,40.000000,1.639344,14.285714,-50.000000,57.142857,44.444444,0.0
2,30.0,53.333333,18.287938,17.647059,-16.666667,66.666667,57.142857,0.0
3,40.0,62.500000,32.203390,28.571429,0.000000,78.947368,62.500000,0.0
4,50.0,68.000000,40.740741,44.827586,15.789474,80.000000,70.000000,0.0
5,60.0,71.666667,46.875000,51.428571,29.166667,83.333333,72.000000,0.0
6,70.0,75.714286,53.388171,58.536585,37.037037,86.486486,75.000000,0.0
7,80.0,76.250000,53.644404,58.695652,38.709677,88.372093,71.875000,0.0
8,90.0,76.666667,53.857422,59.615385,38.235294,90.000000,70.588235,0.0
9,100.0,77.000000,54.247066,60.344828,39.473684,91.071429,70.270270,0.0


## MOA ARF10

In [2]:
from moa.classifiers.meta import AdaptiveRandomForest
from moa.core import Example
from moa.evaluation import BasicClassificationPerformanceEvaluator
from moa.streams import ArffFileStream

arf10 = AdaptiveRandomForest()
arf10.getOptions().setViaCLIString("-s 10")
arf10.setRandomSeed(1)
arf10.prepareForUse()

rtg_2abrupt = ArffFileStream("/Users/gomeshe/Desktop/data/RTG_2abrupt.arff", -1)
rtg_2abrupt.prepareForUse()

acc, wallclock, cpu_time, df = test_train_loop_MOA(rtg_2abrupt, arf10, maxInstances=100000, sampleFrequency=100000)
print(f"{arf10.getClass().getName()}, {acc:.4f}, {wallclock:.4f}, {cpu_time:.4f}")

moa.classifiers.meta.AdaptiveRandomForest, 83.5270, 47.5681, 50.8391


CPU times: user 48.2 s, sys: 193 ms, total: 48.4 s
Wall time: 47.4 s

83.527000

## RIVER ARF10

In [9]:
from river.forest import ARFClassifier
import pandas as pd

rtg_2abrupt_path = '/Users/gomeshe/Desktop/data/RTG_2abrupt.csv'

river_arf10 = ARFClassifier(
    n_models=10,
    max_features=0.60
)

river_rtg_2abrupt = pd.read_csv(rtg_2abrupt_path).to_numpy()

acc, wallclock, cpu_time, df = test_train_loop_RIVER(dataset=river_rtg_2abrupt, model=river_arf10,
                                                     maxInstances=100000, sampleFrequency=100000)
print(f"{river_arf10}, {acc:.4f}, {wallclock:.4f}, {cpu_time:.4f}")

CPU times: user 6min 5s, sys: 3.75 s, total: 6min 8s
Wall time: 6min 9s

Learner:  ARFClassifier  Result:  (0.80196, 368)

## MOA ARF30

# MUST UPDATE THESE OTHER EXPERIMENTS AS WELL

In [28]:
%%time

from moa.classifiers.meta import AdaptiveRandomForest
from moa.core import Example
from moa.evaluation import BasicClassificationPerformanceEvaluator
from moa.streams import ArffFileStream

arf30 = AdaptiveRandomForest()
arf30.getOptions().setViaCLIString("-s 30")
arf30.setRandomSeed(1)
arf30.prepareForUse()

rtg_2abrupt = ArffFileStream("/Users/gomeshe/Desktop/data/RTG_2abrupt.arff", -1)
rtg_2abrupt.prepareForUse()

evaluator = BasicClassificationPerformanceEvaluator()
evaluator.recallPerClassOption.set()
evaluator.prepareForUse()

test_train_loop(rtg_2abrupt, arf30, evaluator, maxInstances=100000, sampleFrequency=1000)

CPU times: user 2min 40s, sys: 508 ms, total: 2min 40s
Wall time: 2min 36s


,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent),Recall for class 0 (percent),Recall for class 1 (percent),Recall for class 2 (percent),Recall for class 3 (percent),Recall for class 4 (percent)
0,1000.0,88.900000,47.856968,58.426966,31.901840,98.685783,38.650307,NaN,NaN,NaN
1,2000.0,89.550000,54.067347,63.008850,37.982196,98.496693,45.400593,NaN,NaN,NaN
2,3000.0,89.500000,55.650876,62.941176,39.539347,98.305768,47.600768,NaN,NaN,NaN
3,4000.0,90.050000,58.071433,64.590747,42.235123,98.399275,49.927431,NaN,NaN,NaN
4,5000.0,90.160000,58.288796,64.806867,42.523364,98.455598,50.000000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
95,96000.0,84.034375,75.052102,69.530644,68.703163,96.978332,70.715536,58.354866,17.525163,90.082503
96,97000.0,83.921649,74.852764,69.387194,68.457245,96.988813,70.620877,58.025742,17.515924,89.952536
97,98000.0,83.835714,74.693075,69.290865,68.253873,97.006299,70.590061,57.839428,17.362887,89.764364
98,99000.0,83.754545,74.535105,69.184933,68.051251,97.024250,70.551270,57.618305,17.163555,89.620224


CPU times: user 2min 40s, sys: 508 ms, total: 2min 40s
Wall time: 2min 36s

83.666000

## RIVER ARF30

In [30]:
%%time

river_arf30 = ARFClassifier(
    n_models=30,
    max_features=0.60
)

river_rtg_2abrupt = pd.read_csv(rtg_2abrupt_path).to_numpy()

result = run_experiment(river_arf30, river_rtg_2abrupt)
print('Learner: ', river_arf30, ' Result: ', result)

Learner:  ARFClassifier  Result:  (0.80162, 1145)
CPU times: user 18min 56s, sys: 11.7 s, total: 19min 7s
Wall time: 19min 6s


CPU times: user 18min 56s, sys: 11.7 s, total: 19min 7s
Wall time: 19min 6s

Learner:  ARFClassifier  Result:  (0.80162, 1145)

## ARF100

In [31]:
%%time

from moa.classifiers.meta import AdaptiveRandomForest
from moa.core import Example
from moa.evaluation import BasicClassificationPerformanceEvaluator
from moa.streams import ArffFileStream

arf100 = AdaptiveRandomForest()
arf100.getOptions().setViaCLIString("-s 100")
arf100.setRandomSeed(1)
arf100.prepareForUse()

rtg_2abrupt = ArffFileStream("/Users/gomeshe/Desktop/data/RTG_2abrupt.arff", -1)
rtg_2abrupt.prepareForUse()

evaluator = BasicClassificationPerformanceEvaluator()
evaluator.recallPerClassOption.set()
evaluator.prepareForUse()

test_train_loop(rtg_2abrupt, arf100, evaluator, maxInstances=100000, sampleFrequency=1000)

CPU times: user 11min 15s, sys: 2.25 s, total: 11min 17s
Wall time: 10min 28s


,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent),Recall for class 0 (percent),Recall for class 1 (percent),Recall for class 2 (percent),Recall for class 3 (percent),Recall for class 4 (percent)
0,1000.0,88.900000,48.185077,58.426966,31.901840,98.566308,39.263804,NaN,NaN,NaN
1,2000.0,89.600000,54.086419,63.185841,38.278932,98.616957,45.103858,NaN,NaN,NaN
2,3000.0,89.466667,55.139083,62.823529,39.347409,98.467124,46.641075,NaN,NaN,NaN
3,4000.0,89.875000,56.827170,63.967972,41.219158,98.550287,48.185776,NaN,NaN,NaN
4,5000.0,90.120000,57.742296,64.663805,42.289720,98.624517,48.948598,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
95,96000.0,83.966667,74.936690,69.401427,68.570437,96.937929,70.651886,58.936262,16.074600,89.859058
96,97000.0,83.862887,74.750844,69.275311,68.341963,96.961477,70.542352,58.642698,16.184134,89.718076
97,98000.0,83.781633,74.598578,69.188120,68.147658,96.983431,70.496975,58.448928,16.197783,89.530439
98,99000.0,83.696970,74.434459,69.075720,67.938021,97.003699,70.428652,58.169527,16.160490,89.392473


CPU times: user 11min 15s, sys: 2.25 s, total: 11min 17s
Wall time: 10min 28s

83.616000

## RIVER ARF100

In [32]:
%%time

river_arf100 = ARFClassifier(
    n_models=100,
    max_features=0.60
)

river_rtg_2abrupt = pd.read_csv(rtg_2abrupt_path).to_numpy()

result = run_experiment(river_arf100, river_rtg_2abrupt)
print('Learner: ', river_arf100, ' Result: ', result)

Learner:  ARFClassifier  Result:  (0.80145, 4384)
CPU times: user 1h 13min 34s, sys: 42.7 s, total: 1h 14min 17s
Wall time: 2h 5s


CPU times: user 1h 13min 34s, sys: 42.7 s, total: 1h 14min 17s
Wall time: 2h 5s

Learner:  ARFClassifier  Result:  (0.80145, 4384)

## ARF100 -j 4

In [8]:
%%time

from moa.classifiers.meta import AdaptiveRandomForest
from moa.core import Example
from moa.evaluation import BasicClassificationPerformanceEvaluator
from moa.streams import ArffFileStream

arf100 = AdaptiveRandomForest()
arf100.getOptions().setViaCLIString("-s 100 -j 4")
arf100.setRandomSeed(1)
arf100.prepareForUse()

rtg_2abrupt = ArffFileStream("/Users/gomeshe/Desktop/data/RTG_2abrupt.arff", -1)
rtg_2abrupt.prepareForUse()

evaluator = BasicClassificationPerformanceEvaluator()
evaluator.recallPerClassOption.set()
evaluator.prepareForUse()

test_train_loop(rtg_2abrupt, arf100, evaluator, maxInstances=100000, sampleFrequency=1000)

CPU times: user 8min 25s, sys: 8.7 s, total: 8min 34s
Wall time: 3min 45s


,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent),Recall for class 0 (percent),Recall for class 1 (percent),Recall for class 2 (percent),Recall for class 3 (percent),Recall for class 4 (percent)
0,1000.0,88.900000,48.185077,58.426966,31.901840,98.566308,39.263804,NaN,NaN,NaN
1,2000.0,89.600000,54.086419,63.185841,38.278932,98.616957,45.103858,NaN,NaN,NaN
2,3000.0,89.466667,55.139083,62.823529,39.347409,98.467124,46.641075,NaN,NaN,NaN
3,4000.0,89.875000,56.827170,63.967972,41.219158,98.550287,48.185776,NaN,NaN,NaN
4,5000.0,90.120000,57.742296,64.663805,42.289720,98.624517,48.948598,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
95,96000.0,83.966667,74.936690,69.401427,68.570437,96.937929,70.651886,58.936262,16.074600,89.859058
96,97000.0,83.862887,74.750844,69.275311,68.341963,96.961477,70.542352,58.642698,16.184134,89.718076
97,98000.0,83.781633,74.598578,69.188120,68.147658,96.983431,70.496975,58.448928,16.197783,89.530439
98,99000.0,83.696970,74.434459,69.075720,67.938021,97.003699,70.428652,58.169527,16.160490,89.392473


## ARF100 -j 8

In [7]:
%%time

from moa.classifiers.meta import AdaptiveRandomForest
from moa.core import Example
from moa.evaluation import BasicClassificationPerformanceEvaluator
from moa.streams import ArffFileStream

arf100 = AdaptiveRandomForest()
arf100.getOptions().setViaCLIString("-s 100 -j 8")
arf100.setRandomSeed(1)
arf100.prepareForUse()

rtg_2abrupt = ArffFileStream("/Users/gomeshe/Desktop/data/RTG_2abrupt.arff", -1)
rtg_2abrupt.prepareForUse()

evaluator = BasicClassificationPerformanceEvaluator()
evaluator.recallPerClassOption.set()
evaluator.prepareForUse()

test_train_loop(rtg_2abrupt, arf100, evaluator, maxInstances=100000, sampleFrequency=1000)

CPU times: user 9min 43s, sys: 12.7 s, total: 9min 55s
Wall time: 4min 18s


,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent),Recall for class 0 (percent),Recall for class 1 (percent),Recall for class 2 (percent),Recall for class 3 (percent),Recall for class 4 (percent)
0,1000.0,88.900000,48.185077,58.426966,31.901840,98.566308,39.263804,NaN,NaN,NaN
1,2000.0,89.600000,54.086419,63.185841,38.278932,98.616957,45.103858,NaN,NaN,NaN
2,3000.0,89.466667,55.139083,62.823529,39.347409,98.467124,46.641075,NaN,NaN,NaN
3,4000.0,89.875000,56.827170,63.967972,41.219158,98.550287,48.185776,NaN,NaN,NaN
4,5000.0,90.120000,57.742296,64.663805,42.289720,98.624517,48.948598,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
95,96000.0,83.966667,74.936690,69.401427,68.570437,96.937929,70.651886,58.936262,16.074600,89.859058
96,97000.0,83.862887,74.750844,69.275311,68.341963,96.961477,70.542352,58.642698,16.184134,89.718076
97,98000.0,83.781633,74.598578,69.188120,68.147658,96.983431,70.496975,58.448928,16.197783,89.530439
98,99000.0,83.696970,74.434459,69.075720,67.938021,97.003699,70.428652,58.169527,16.160490,89.392473


CPU times: user 9min 43s, sys: 12.7 s, total: 9min 55s
Wall time: 4min 18s

83.616000	

## ARF100 -j 2

In [8]:
%%time

from moa.classifiers.meta import AdaptiveRandomForest
from moa.core import Example
from moa.evaluation import BasicClassificationPerformanceEvaluator
from moa.streams import ArffFileStream

arf100 = AdaptiveRandomForest()
arf100.getOptions().setViaCLIString("-s 100 -j 2")
arf100.setRandomSeed(1)
arf100.prepareForUse()

rtg_2abrupt = ArffFileStream("/Users/gomeshe/Desktop/data/RTG_2abrupt.arff", -1)
rtg_2abrupt.prepareForUse()

evaluator = BasicClassificationPerformanceEvaluator()
evaluator.recallPerClassOption.set()
evaluator.prepareForUse()

test_train_loop(rtg_2abrupt, arf100, evaluator, maxInstances=100000, sampleFrequency=1000)

CPU times: user 9min 42s, sys: 8.26 s, total: 9min 50s
Wall time: 6min 20s


,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent),Recall for class 0 (percent),Recall for class 1 (percent),Recall for class 2 (percent),Recall for class 3 (percent),Recall for class 4 (percent)
0,1000.0,88.900000,48.185077,58.426966,31.901840,98.566308,39.263804,NaN,NaN,NaN
1,2000.0,89.600000,54.086419,63.185841,38.278932,98.616957,45.103858,NaN,NaN,NaN
2,3000.0,89.466667,55.139083,62.823529,39.347409,98.467124,46.641075,NaN,NaN,NaN
3,4000.0,89.875000,56.827170,63.967972,41.219158,98.550287,48.185776,NaN,NaN,NaN
4,5000.0,90.120000,57.742296,64.663805,42.289720,98.624517,48.948598,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
95,96000.0,83.966667,74.936690,69.401427,68.570437,96.937929,70.651886,58.936262,16.074600,89.859058
96,97000.0,83.862887,74.750844,69.275311,68.341963,96.961477,70.542352,58.642698,16.184134,89.718076
97,98000.0,83.781633,74.598578,69.188120,68.147658,96.983431,70.496975,58.448928,16.197783,89.530439
98,99000.0,83.696970,74.434459,69.075720,67.938021,97.003699,70.428652,58.169527,16.160490,89.392473


CPU times: user 9min 42s, sys: 8.26 s, total: 9min 50s
Wall time: 6min 20s

83.616000

## ARF100 -j 5

In [9]:
%%time

from moa.classifiers.meta import AdaptiveRandomForest
from moa.core import Example
from moa.evaluation import BasicClassificationPerformanceEvaluator
from moa.streams import ArffFileStream

arf100 = AdaptiveRandomForest()
arf100.getOptions().setViaCLIString("-s 100 -j 5")
arf100.setRandomSeed(1)
arf100.prepareForUse()

rtg_2abrupt = ArffFileStream("/Users/gomeshe/Desktop/data/RTG_2abrupt.arff", -1)
rtg_2abrupt.prepareForUse()

evaluator = BasicClassificationPerformanceEvaluator()
evaluator.recallPerClassOption.set()
evaluator.prepareForUse()

test_train_loop(rtg_2abrupt, arf100, evaluator, maxInstances=100000, sampleFrequency=1000)

CPU times: user 9min 39s, sys: 8.91 s, total: 9min 48s
Wall time: 4min 37s


,classified instances,classifications correct (percent),Kappa Statistic (percent),Kappa Temporal Statistic (percent),Kappa M Statistic (percent),Recall for class 0 (percent),Recall for class 1 (percent),Recall for class 2 (percent),Recall for class 3 (percent),Recall for class 4 (percent)
0,1000.0,88.900000,48.185077,58.426966,31.901840,98.566308,39.263804,NaN,NaN,NaN
1,2000.0,89.600000,54.086419,63.185841,38.278932,98.616957,45.103858,NaN,NaN,NaN
2,3000.0,89.466667,55.139083,62.823529,39.347409,98.467124,46.641075,NaN,NaN,NaN
3,4000.0,89.875000,56.827170,63.967972,41.219158,98.550287,48.185776,NaN,NaN,NaN
4,5000.0,90.120000,57.742296,64.663805,42.289720,98.624517,48.948598,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
95,96000.0,83.966667,74.936690,69.401427,68.570437,96.937929,70.651886,58.936262,16.074600,89.859058
96,97000.0,83.862887,74.750844,69.275311,68.341963,96.961477,70.542352,58.642698,16.184134,89.718076
97,98000.0,83.781633,74.598578,69.188120,68.147658,96.983431,70.496975,58.448928,16.197783,89.530439
98,99000.0,83.696970,74.434459,69.075720,67.938021,97.003699,70.428652,58.169527,16.160490,89.392473
